In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
#import keras.backend as K
from flowers_utils import *

In [3]:
from keras.layers import Add, Input, Dense, ZeroPadding2D, Activation, Conv2D, BatchNormalization, MaxPooling2D, Flatten
from keras.initializers import glorot_uniform
from keras.models import Model
from keras.optimizers import Adam

In [4]:
train_it= load_images()

Found 3670 images belonging to 5 classes.


In [5]:
train_it.next()[0][0].shape

(64, 64, 3)

In [6]:
train_it.next()[1].shape[1]  ### classes size

5

In [7]:
def identity_block(a_prev, filters, stride, kernel_size):
    
    F1, F2 =filters              ###  F2 must be equal to c    if a_prev has shape(n,n,c)
    
    s=stride
    
    k= kernel_size
    
    a_input=a_prev
    
    X=Conv2D(filters=F1, kernel_size=(1,1), strides=(1,1), padding='valid', kernel_initializer=glorot_uniform(seed=0))(a_prev)
    
    X=BatchNormalization(axis=3)(X)   ### batchnormalization through channels.....................
    
    X=Activation('relu')(X)
    
    X=Conv2D(filters=F2, kernel_size=(k,k), strides=(1,1), padding='same', kernel_initializer=glorot_uniform(seed=0))(X)
    
    X=BatchNormalization(axis=3)(X)
    
    X=Add()([a_input, X])
    
    X=Activation('relu')(X)
    
    return X
    

In [8]:
def convolutional_block(a_prev, stride, filters, kernel_size):
    
    
    F1, F2 =filters
    
    k=kernel_size
    s=stride
    
    a_input=a_prev
    
   
    X=Conv2D(filters=F1, kernel_size=(1,1), strides=(s,s), padding='valid', kernel_initializer=glorot_uniform(seed=0))(a_prev)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    
    X=Conv2D(filters=F2, kernel_size=(k,k), strides=(1,1), padding='same', kernel_initializer=glorot_uniform(seed=0))(X)
    X=BatchNormalization(axis=3)(X)
    
    
    a_input= Conv2D(filters=F2, kernel_size=(1,1), strides=(s,s), padding='valid', kernel_initializer=glorot_uniform(seed=0))(a_input)
    a_input=BatchNormalization(axis=3)(a_input)
    
    X=Add()([a_input, X])
    X=Activation('relu')(X)
    
    return X
    

In [9]:
def model(input_shape, filters, stride, kernel_size, classes_size):
    
    
    X_input=Input(shape=input_shape)
    
    X=ZeroPadding2D(padding=(3,3))(X_input)
    
    X=Conv2D(filters=64 , kernel_size=(5,5), padding='valid', strides=(2,2), kernel_initializer= glorot_uniform (seed=0))(X)
    X=BatchNormalization(axis=3)(X)
    X=Activation('relu')(X)
    
    X=MaxPooling2D(pool_size=(3,3), strides=(2,2))(X)
    
    X= convolutional_block(X, stride, filters, kernel_size)
    X= identity_block(X, filters, stride, kernel_size)
    X= identity_block(X, filters, stride, kernel_size)
    
    X= convolutional_block(X, stride, filters, kernel_size)
    X= identity_block(X, filters, stride, kernel_size)
    X= identity_block(X, filters, stride, kernel_size)
    X= identity_block(X, filters, stride, kernel_size)
    
    
    X=Flatten()(X)
    
    #X=Dense(units=128, activation='relu')(X)
    
    X=Dense(units=classes_size, activation='softmax')(X)
    
    model=Model(inputs=X_input, outputs=X)
    
    return model
    

In [10]:
model=model(input_shape=(64,64,3), filters=[3,5], stride=2, kernel_size=3, classes_size=5)

In [11]:
opt=Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model.fit_generator(train_it, steps_per_epoch=32, epochs=15)

Epoch 1/15
32/32 [==============================] - 13s - loss: 2.1433 - acc: 0.2588    
Epoch 2/15
32/32 [==============================] - 13s - loss: 1.6003 - acc: 0.2988    
Epoch 3/15
32/32 [==============================] - 13s - loss: 1.5022 - acc: 0.3496    
Epoch 4/15
32/32 [==============================] - 14s - loss: 1.4647 - acc: 0.3475    
Epoch 5/15
32/32 [==============================] - 15s - loss: 1.3621 - acc: 0.4160    
Epoch 6/15
32/32 [==============================] - 16s - loss: 1.3162 - acc: 0.4473    
Epoch 7/15
32/32 [==============================] - 14s - loss: 1.3460 - acc: 0.4248    
Epoch 8/15
32/32 [==============================] - 13s - loss: 1.2207 - acc: 0.4580    
Epoch 9/15
32/32 [==============================] - 13s - loss: 1.1977 - acc: 0.4775    
Epoch 10/15
32/32 [==============================] - 13s - loss: 1.1810 - acc: 0.4971    
Epoch 11/15
32/32 [==============================] - 13s - loss: 1.1441 - acc: 0.5067    
Epoch 12/15
32/32 [